# Part A

In [67]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium
print("loaded library")

loaded library


In [68]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [69]:
ls_canada = pd.read_html(url)
df_canada = ls_canada[0]
df_canada_processed = df_canada[df_canada.Borough != "Not assigned"]
df_canada_processed.head()
df_canada_processed.sort_values(by=['Postcode'], inplace = True)
df_canada_processed.reset_index(inplace = True)
del df_canada_processed['index']
df_canada_processed.shape
df= df_canada_processed
tempDf =df.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index().set_index('Postcode')

#droping rows and joining above dataframes 
df = df.drop('Neighbourhood', axis=1).drop_duplicates().set_index('Postcode')
df = df.join(tempDf)
print(df)


                   Borough                                      Neighbourhood
Postcode                                                                     
M1B            Scarborough                                      Rouge,Malvern
M1C            Scarborough               Port Union,Rouge Hill,Highland Creek
M1E            Scarborough                    Guildwood,Morningside,West Hill
M1G            Scarborough                                             Woburn
M1H            Scarborough                                          Cedarbrae
M1J            Scarborough                                Scarborough Village
M1K            Scarborough          East Birchmount Park,Ionview,Kennedy Park
M1L            Scarborough                      Golden Mile,Oakridge,Clairlea
M1M            Scarborough      Cliffcrest,Scarborough Village West,Cliffside
M1N            Scarborough                         Cliffside West,Birch Cliff
M1P            Scarborough  Wexford Heights,Dorset Park,Scarboro

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [70]:
df.shape
   
    

(103, 2)

# PART B

In [71]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
loc_table = df.join(geo_data.set_index('Postal Code'))
loc_table = loc_table.sample(frac=1).reset_index(drop=True)
loc_table.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,York Mills West,43.752758,-79.400049
1,Etobicoke,"Richview Gardens,Kingsview Village,St. Phillip...",43.688905,-79.554724
2,Downtown Toronto,Christie,43.669542,-79.422564
3,York,"Silverthorn,Mount Dennis,Keelesdale,Del Ray",43.691116,-79.476013
4,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999


# PART C

In [81]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [77]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {0}, {1}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [82]:
map_geo = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(loc_table['Latitude'], loc_table['Longitude'], loc_table['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_geo)  
    
map_geo